**Draft Model for Mice Steep Stage Analysis**

In [1]:
import random
import numpy as np
import pandas as pd
import os
import re
import copy
import matplotlib.pyplot as plt

import torch
from torch import optim, nn
from torchvision import transforms, datasets, models

from collections import OrderedDict

import torch.utils.data as utils

import tkinter as tk #tk for file dialog (requires Jinja2!!!)
from tkinter import filedialog #tkinter for file dialog

import re #regex for parsing
from os.path import exists


In [6]:
from inputMassager import *
inputHandler = inputMassager()

filepath = ("CHD801FR_20221123_normal.txt")

#Period Size Variable : effects CNN architecture
periodSize = 200

#makePeriodFromTxt(self, filepath, periodSize, maxPeriods=None):
periods = inputHandler.makePeriodFromTxt(filepath, periodSize, 2000)


# target = regan and audreys code ---> This is what will go into the data loader
filepath2 = ("CHDCtrl1_CHD801FR_normal_annotatedl.txt")

timestamps = find_time_labels(df, filepath2)#2d list of stop times and corresponding labels from annotated data
labels = label_dataframe(periods, timestamps)#list of the corresponding label for each period based on timestamps


TclError: ignored

Makes 3d tensor out of period data for either the eeg(c1) or emg(c2) column

the tensor is size [number of samples, 1, period_size], it needs to be this shape becasue the 2nd dimesion of an input tensor is interpreted as the number of input channels by convoluntional layers

when you concatanate the eeg samples and emg samples, it becomes a [number of samples, 2, period_size] tensor and emg becomes another channel

In [ ]:
def getColTensor(col):
    #The tensor has to be 3 dimesional with the second dimesion equal to one in order to pass it through a 1d convolutional layer
    col_tensor = torch.zeros((len(periods[col].values), 1, periodSize))

    for i in range(len(periods[col].values)):
        col_tensor[i, 0] = torch.tensor(periods[col].values[i])
    
    return col_tensor

eeg_samples = getColTensor()

In [ ]:
eeg_samples = getColTensor("c1")
emg_samples =  getColTensor("c2")

both_samples = torch.cat((eeg_samples, emg_samples), 1)

This is supposed to get only the first 2000 appearances of rem, non-rem and wake data (for a total of 6000) so that we don't just have a ton of one and we can get a little better idea of how the model might actually do but it'll mess up the order near the end becuase once it's hit for instance 2000 wake states, it'll stop getting those so we'll lose the timing context, i'm ot sure if it actully useful but if you do use, i think you probably wan to set shuffle to true

In [ ]:
def equalRepresentation():
    
    count1, count2, count3 = 0, 0, 0
    f_labels = []
    f_periods = [[],[]]

    for i in range(len(labels)):
        if labels[i] == 1 and count1 < 2000:
            f_labels.append(labels[i])
            f_periods[0].append(periods["c1"][i])
            f_periods[1].append(periods["c2"][i])
            count1+= 1

        if labels[i] == 2 and count2 < 2000:
            f_labels.append(labels[i])
            f_periods[0].append(periods["c1"][i])
            f_periods[1].append(periods["c2"][i])
            count2+= 1

        if labels[i] == 3 and count3 < 2000:
            f_labels.append(labels[i])
            f_periods[0].append(periods["c1"][i])
            f_periods[1].append(periods["c2"][i])
            count3+= 1

    return f_labels, f_periods

In [ ]:
f_labels, f_periods = equalRepresentation()

getColTensor for equal representation samples

In [ ]:
def equal_getColTensor(col = 0):
    col_tensor = torch.zeros((len(f_periods[0]), 1, 200))

    for i in range(len(f_periods[0])):
        col_tensor[i, 0] = torch.tensor(f_periods[col][i])
    
    return col_tensor

equal_eeg_samples = equal_getColTensor(0)
equal_emg_samples = equal_getColTensor(1)

equal_both_samples = torch.cat((eeg_samples, emg_samples), 1)
equal_labels = torch.tensor(f_labels)

Make the train and validation data loaders  

In [ ]:
ds = torch.utils.data.TensorDataset(both_samples, torch.tensor(labels))

train_size = int(len(ds) *.80)
val_size = len(ds) - int(len(ds) *.80)

train_dataset, val_dataset = torch.utils.data.random_split(ds, [train_size, val_size])


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle = False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=100, shuffle=False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Creating class for Model**

In [ ]:
import torch.nn.functional as F
from torch import nn, optim


class CNN(nn.Module):
    #takes just one input for the number of input channels, right now if you're using both eeg and emg it's 2 and if youre just using one of them, its 1    
    def __init__(self, in_vars):
        super(CNN, self).__init__()

        #out_channels and kernal size are random        
        self.conv1 = nn.Conv1d(in_channels=in_vars, out_channels= 100, kernel_size=2)

        #the second number should be the number of classifications, it's 5 now becuase the clasifcications range from 1-4 but it should be 4 so artificat should eventually be 0  
        self.fc1 = nn.Linear(9900, 5)


    def forward(self, x, verbose=False):

        x = self.conv1(x)
        x = F.relu(x)
        #print(x.size())

        x = F.max_pool1d(x, kernel_size=2)#kernal size is random
        #print(x.size())
        
        x = x.flatten(1)
        #print(x.size())

        x = self.fc1(x)
        x = F.log_softmax(x, dim=1)#softmax is random

        return x

In [2]:
def train_model(epochs, model):
    model.train()#set model to training mode
    loss_fun = nn.CrossEntropyLoss()#define a loss function object

    for epoch in range(epochs):

      for batch_idx, (data, target) in enumerate(train_loader):
          data, target = data.to(device), target.to(device)
          
          optimizer.zero_grad()
          output = model(data)
          loss = loss_fun(output,target)
          loss.backward()
          optimizer.step()
          if batch_idx % 100 == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
basic_cnn = CNN(2)
basic_cnn.to(device)
  
optimizer = torch.optim.SGD(basic_cnn.parameters(), lr=.01)
train_model(10, basic_cnn)

In [ ]:
def evaluate_model(model, dataloader, is_test=False):
  #Evaluation

  # Set model to evaluation mode
  model.eval()

  with torch.no_grad():
    correct = 0
    loss = 0

    criterion = torch.nn.CrossEntropyLoss(reduction='none')

    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        
        loss += torch.sum(criterion(outputs, target)).item()
        
        pred = outputs.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    loss /= len(dataloader.dataset)
    accuracy = 100. * correct / len(dataloader.dataset)
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        "Test" if is_test else "Validation",
        loss, correct, len(dataloader.dataset),
        accuracy))
  # Set model back to training mode
  model.train()

In [ ]:
evaluate_model(basic_cnn, val_loader, is_test=True)